# Console Session Report (Automatically Generated)
**Created by:** Mark
**Date:** August 08, 2024
**Model Version:** models/gemini-pro

**Session outcome:** *Call rag_chain.stream() and print the results*

 **Importing necessary libraries**


In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from langchain import hub

os.environ['GOOGLE_API_KEY'] = open('google_api_key.txt').read()

In [6]:


bs4_strainer = bs4.SoupStrainer()
loader = WebBaseLoader(
    web_paths=("https://degrees.apps.asu.edu/minors/major/ASU00/BABDACERT/applied-business-data-analytics?init=false&nopassive=true",),
    bs_kwargs={"parse_only": bs4_strainer},
)
data = loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits__ = text_splitter.split_documents(data)

 **Creating a vector store using Chroma**


In [16]:
vectorstore = Chroma.from_documents(documents=all_splits__, embedding=HuggingFaceEmbeddings())

 **Creating a retriever using the vector store**


In [17]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

 **Creating a RAG chain**


In [18]:
model = ChatGoogleGenerativeAI(model='gemini-1.5-flash')
prompt = hub.pull("rlm/rag-prompt")
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [22]:
retriever.invoke('What do students learn in the Applied Business Data Analytics program at ASU?')

[Document(metadata={'description': ' ', 'language': 'en', 'source': 'https://degrees.apps.asu.edu/minors/major/ASU00/BABDACERT/applied-business-data-analytics?init=false&nopassive=true', 'start_index': 2847, 'title': 'Applied Business Data Analytics - Certificate | Degree Details | ASU Degree Search'}, page_content="Enrollment requirements\n▲\n\n\n\n\n\n\n\nASU degree-seeking studentsThe applied business data analytics certificate is open to all ASU students who are not pursuing the applied business and technology solutions or entrepreneurial leadership degree programs.No more than six credit hours may be shared between the certificate and other degree programs, minors or other certificates.Nondegree-seeking studentsThis certificate is available through the Pathways for the Future program. Applicants without a bachelor's degree are eligible to apply for, and receive, this certificate, through the Pathways for the Futures program. Applicants who already hold a bachelor's degree should a

 **Streaming the RAG chain to get the answer**


In [26]:
for chunk in rag_chain.stream("What do students learn in the Applied Business Data Analytics program?"):
    print(chunk, end="", flush=True)

Students in the Applied Business Data Analytics program learn how to manage and analyze large datasets using sophisticated but user-friendly computer-based tools. They gain expertise in predictive analytics, big data techniques, and visualization to make informed decisions about projects, processes, and tasks. This program emphasizes critical thinking skills to develop evidence-based benchmarks for high-quality outcomes. 
